In [104]:
import gmsh
import sys
import math

# Before using any functions in the Python API, Gmsh must be initialized:
gmsh.initialize()

gmsh.clear()
# Next we add a new model named "t1" (if gmsh.model.add() is not called a new
# unnamed model will be created on the fly, if necessary):
gmsh.model.add("HEU-MET-FAST-003=1")

Info    : Clearing all models and views...
Info    : Done clearing all models and views


In [105]:
# Define geometry parameters (example: two spheres)
inner_radius = 6.1535
outer_radius = 18.8535

In [106]:
inner_sphere = gmsh.model.occ.addSphere(0, 0, 0, inner_radius)
outer_sphere = gmsh.model.occ.addSphere(0, 0, 0, outer_radius)

In [107]:
cutResult = gmsh.model.occ.cut([(3, outer_sphere)], [(3, inner_sphere)], removeTool=False)

print("Cut result:", cutResult)
# Synchronize the CAD kernel with the Gmsh model
gmsh.model.occ.synchronize()
# Identify the volumes: we expect two volumes (the inner sphere and the shell)
volEntities = gmsh.model.getEntities(3)
print("Volumes:", volEntities)
shell_volume = None
for (dim, tag) in volEntities:
    if tag != inner_sphere:
        shell_volume = tag
        break
        
if shell_volume is None:
    raise RuntimeError("Could not identify the shell volume.")
# Define physical groups for the inner sphere and shell

Cut result: ([(3, 2)], [[(3, 2)], []])                                                                                    
Volumes: [(3, 1), (3, 2)]


In [108]:
# Define physical groups for the inner sphere and shell
gmsh.model.addPhysicalGroup(3, [inner_sphere], tag=1)
gmsh.model.setPhysicalName(3, 1, "Inner")
gmsh.model.addPhysicalGroup(3, [shell_volume], tag=2)
gmsh.model.setPhysicalName(3, 2, "Outer")
# Create a MathEval field to control the mesh size based on z.
fieldTag = gmsh.model.mesh.field.add("MathEval")
gmsh.model.mesh.field.setString(fieldTag, "F", "2.5")
# Use this field as the background mesh field.
gmsh.model.mesh.field.setAsBackgroundMesh(fieldTag)

In [109]:
# Generate the 3D mesh
gmsh.model.mesh.generate(3)
# Save mesh in VTK format
gmsh.write("two_spheres.vtk")
gmsh.write("Meshes/case_4.msh")
if "-nopopup" not in sys.argv:
    gmsh.fltk.run()
    gmsh.finalize()

Info    : Meshing 1D...
Info    : [ 20%] Meshing curve 2 (Circle)
Info    : [ 70%] Meshing curve 5 (Circle)
Info    : Done meshing 1D (Wall 0.000244258s, CPU 0.000371s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Sphere, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 2 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0376819s, CPU 0.039355s)
Info    : Meshing 3D...
Info    : 3D Meshing 2 volumes with 1 connected component
Info    : Tetrahedrizing 1000 nodes...
Info    : Done tetrahedrizing 1008 nodes (Wall 0.00607736s, CPU 0.005342s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0134533s, CPU 0.013059s)
Info    : Found volume 2
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 2.85343 (nodes removed 0 0)
Info    : It. 500 - 498 nodes created - worst tet radius 1.19857 (nodes removed 0 2)
Inf